# QTensorAI Tutorial

To start using QTensorAI for your own hybrid quantum-classical neural network research, you need to do a few things.
1. Write your own circuit composer.
2. Write your own hybrid pytorch neural network module.

The circuit composer is what you use to create your circuit, and the hybrid module is what you use to integrate with a machine learning pipeline.

We have built base classes of circuit composers and hybrid modules that allows you to focus on the science of your study, rather than the implementation details of our library.

## Custom Circuit Composer

Let's say we want to create a straight forward quantum neural network as is described by Abbas et. al. in arXiv:2011.00027. We will not focus on the circuit, but on how you can use our library. First, let us import the class that facilitates your creation of a custom circuit composer. This class is the `ParallelComposer` class.

In [1]:
from qtensor_ai import ParallelComposer

/home/minzhaoliu/.conda/envs/qtensor_ai/lib/python3.11/site-packages/tqdm-4.65.0-py3.11.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Then, we can create our own composer inheriting from the `ParallelComposer` class. This composer is also provided in `examples/Custom_Circuit_Composers.py`. The comments in this notebook are made with a narrative flow and is best to read without skipping.

In [2]:
class QNNComposer(ParallelComposer):
    
    '''For initialization, n_qubits is always needed since the ParallelComposer class requires that
    The other parameters are specific to this example and are not important.'''
    def __init__(self, n_qubits, n_layers, higher_order=False):
        self.n_layers = n_layers
        self.higher_order = higher_order
        super().__init__(n_qubits)
    
    '''This creates a layer of Hadamard gates. To apply any gates,
    use self.apply_gate(self.operators.gate, *qubits, **parameters).
    There can be mulple qubits or parameters, depending on the gate type.
    Other functions below are logics to add gates. We can ignore them for now.'''
    def layer_of_Hadamards(self):
        for q in self.qubits:
            self.apply_gate(self.operators.H, q)
    
    def entangling_layer(self):
        for i in range(self.n_qubits//2):
            control_qubit = self.qubits[2*i]
            target_qubit = self.qubits[2*i+1]
            self.apply_gate(self.operators.cX, control_qubit, target_qubit)
        for i in range((self.n_qubits+1)//2-1):
            control_qubit = self.qubits[2*i+1]
            target_qubit = self.qubits[2*i+2]
            '''For example, you can put two qubits for CNOT (cX) gates.'''
            self.apply_gate(self.operators.cX, control_qubit, target_qubit)
        control_qubit = self.qubits[-1]
        target_qubit = self.qubits[0]
    
    def encoding_circuit(self, data):
        self.layer_of_Hadamards()
        for i, qubit in enumerate(self.qubits):
            self.apply_gate(self.operators.ZPhase, qubit, alpha=data[:, i])
        if self.higher_order:
            for i in range(self.n_qubits):
                for j in range(i+1, self.n_qubits):
                    control_qubit = self.qubits[i]
                    target_qubit = self.qubits[j]
                    self.apply_gate(self.operators.cX, control_qubit, target_qubit)
                    self.apply_gate(self.operators.ZPhase, target_qubit, alpha=data[:, i]*data[:, j])
                    self.apply_gate(self.operators.cX, control_qubit, target_qubit)
    
    def variational_layer(self, layer, layer_params):
        for i in range(self.n_qubits):
            qubit = self.qubits[i]
            '''For an RY (YPhase) gate, there is one qubit,
            and a alpha parameter which is a torch.Tensor of size (n_batch, 1).'''
            self.apply_gate(self.operators.YPhase, qubit, alpha=layer_params[:, i])
     
    def cost_operator(self):
        for qubit in self.qubits:
            self.apply_gate(self.operators.Z, qubit)

    def forward_circuit(self, data, params):
        self.encoding_circuit(data)
        self.entangling_layer()
        for layer in range(self.n_layers):
            self.variational_layer(layer, params[:, :, layer])
            self.entangling_layer()
    '''The detailed inputs of apply_gate will depend on the gate, and you can implement
    custom gates as well. For those, you could ask for any fancy input parameters.

    Moving on, with all the functions for building the circuit ready, we can implement
    required functions. Specifically, any custom composers must have update_full_circuit
    and name.

    This function builds circuit whose first amplitude is the expectation value of
    the measured circuit w.r.t. the cost_operator.
    This function needs to return the circuit (a list) whose first amplitude you want to simulate.'''
    def updated_full_circuit(self, **parameters):
        data = parameters['data']
        params = parameters['params']
        '''All the apply_gate operations actually appends the gates to self.builder.circuit.
        Hence, after we call the functions that builds the circuits, wee need to fetch the
        circuit from self.builder. We will also need to clean the builder up from time to time.'''
        self.builder.reset() # Clear builder.circuit
        self.forward_circuit(data, params) # Set builder.circuit to the forward circuit according to data and params
        self.cost_operator() # Add the cost operators to builder.circuit
        first_part = self.builder.circuit # Extract builder.circuit at this stage for later use
        self.builder.reset() # Clear builder.circuit
        self.forward_circuit(data, params) # Set builder.circuit to the forward circuit according to data and params
        self.builder.inverse() # Change builder.circuit to it's reverse, which is the forward circuit in reverse
        second_part = self.builder.circuit # Extract the inverse circuit
        self.builder.reset() # Clear builder circuit
        '''The final circuit is forward + cost + inverse.
        The first amplitude is the expectation value of the cost operator
        for the forward circuit initialized with the 0 state.'''
        return first_part + second_part
    '''Although this function returns a circuit, we should not call this function in general.
    This is because the parent class uses this function for the produce_circuit method to
    generate the circuit and tensors on the GPU behind the scene.'''

    '''This function returns the name of the circuit composer'''
    def name(self):
        return 'QNN'

To recap, you need to initialize the `ParallelComposer` class with `n_qubits`, call `self.apply_gate` to add gates to `self.builder.circuit`, and create the `update_full_circuit` method which returns the circuit whose first amplitude you want to simulate. Finally, create the `name` method.

## Custom Hybrid Module

Now, let us move on to creating a custom hybrid module that can interface with a machine learning pipeline. First, import the `HybridModule` class.

In [4]:
from qtensor_ai import HybridModule, DefaultOptimizer
import torch
import torch.nn as nn

We also include the `DefaultOptimizer` class to put in as the default choice for the optimizer. We will create a drop-in replacement of a classical fully connected layer called `QNN`. This can also be found in `examples/Custom_Modules.py`.

In [5]:
'''This is a drop-in replacement of linear layers.
The number of input features is the number of qubits.
Each output feature is computed by an independently parameterized circuit'''
class QNN(HybridModule):
    
    def __init__(self, in_features, out_features, variational_layers=1, higher_order=False, optimizer=DefaultOptimizer()):
                
        '''Initializing module parameters
        The variable circuit_name is needed for initialization of the parent class HybridModule
        All the other attributes are unique to this circuit and we can ignore.'''
        circuit_name = 'n_{}_l_{}'.format(in_features, variational_layers)
        self.higher_order = higher_order
        self.in_features = in_features
        self.out_features = out_features
        self.variational_layers = variational_layers
        self.higher_order = higher_order
        
        '''Define the circuit composer and initialize the hybrid module.
        The composer is the custom composer we just defined.'''
        composer = QNNComposer(in_features, variational_layers, higher_order=higher_order)
        super(QNN, self).__init__(circuit_name=circuit_name, composer=composer, optimizer=optimizer)

        '''self.weight are model weights. Weights must be defined after super().__init__()'''
        self.weight = nn.Parameter(torch.randn(out_features, in_features, variational_layers, dtype=torch.float32))


    def forward(self, x):
        '''These lines of code are trying to manipulate the array to give the right input to the simulation.
        The circuits with different parameters will be simulated in a batch parallel manner.
        The 0-th dimension is the parallel batch dimension.'''
        n_batch = x.shape[0] # (n_batch, in_features)
        # Because for multiple output features, the parallelism is in the batch dimension as well as the
        # output feature size dimension, we need to make the 0-th dimension of size out_features*n_batch
        x = x.repeat(self.out_features, 1) # (out_features*n_batch, in_features)
        params = self.weight.unsqueeze(1) # (out_features, 1, in_features, variational_layers)
        params = params.expand(-1, n_batch, -1, -1) # (out_features, n_batch, in_features, variational_layers)
        params = params.reshape(self.out_features*n_batch, self.in_features, self.variational_layers) # (out_features*n_batch, in_features, variational_layers)

        '''The actual simulation must be run by calling the parent_forward method of the parent class. 
        The parameters should be the same parameters as those accepted by the circuit composer'''
        out = self.parent_forward(data=x, params=params) # (out_features*n_batch)
        # Reshaping the outputs
        out = torch.real(out) # (out_features*n_batch)
        out = out.reshape(self.out_features, n_batch) # (out_features, n_batch)
        out = out.permute(1, 0) # (n_batch, out_features)
        return out

To recap, we need to have a `circuit_name`, initialize the `HybridModule` parent class with your circuit name, composer and optimizer, and only after super init create any model weights. Finally, to run the simulation, `self.parent_forward` must be used in `forward`.

We can now build a quantum convolutional module with the QNN module.

In [6]:
from math import floor

In [33]:
'''This is an example for 1D convolution. The filter is replaced with the QNN.'''
class QConv2D(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, variational_layers=1, higher_order=False, optimizer=DefaultOptimizer(), dilation=(1, 1), padding=(0, 0), stride=(1, 1)):
        super().__init__()
                
        '''Initializing module parameters'''
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.n_qubits = in_channels * kernel_size[0] * kernel_size[1]
        
        '''Defining unfold operation for convolution'''
        self.dilation = dilation
        self.padding = padding
        self.stride = stride
        self.unfold = nn.Unfold(kernel_size=kernel_size, dilation=dilation, padding=padding, stride=stride)
        
        '''Defining multichannel filter to be convolved'''
        self.kernel = QNN(self.n_qubits, out_channels, variational_layers, higher_order, optimizer)
        

    '''This function transforms batched, multichannel sequences into parallel values of convolution kernel inputs'''
    def memory_strided_im2col(self, x):
        # x has dimension (n_batch, in_channels, length1, length2)
        out = self.unfold(x)
        out = torch.transpose(out, 1, 2)
        # out has dimension (n_batch, L, kernel_size[0]*kernel_size[1]*in_channels=n_qubits)
        return out
    
    def forward(self, x):
        kernel_size = self.kernel_size
        padding = self.padding
        dilation = self.dilation
        stride = self.stride
        n_batch = x.size(0) # (n_batch, in_channels, length1, length2)
        h_in = x.size(2)
        w_in = x.size(3)
        x = self.memory_strided_im2col(x) # (n_batch, L, kernel_size[0]*kernel_size[1]*in_channels=n_qubits)
        x = x.reshape(-1, kernel_size[0]*kernel_size[1]*self.in_channels) # (n_batch*L, kernel_size*in_channels=n_qubits)
        output = self.kernel(x) # (n_batch*L, out_channels)  
        output = output.reshape(n_batch, -1, self.out_channels) # (n_batch, L, out_channels) 
        output = output.transpose(1, 2) # (n_batch, out_channels, L)
        h_out = floor((h_in + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1)
        w_out = floor((w_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1)
        output = torch.nn.functional.fold(output, (h_out, w_out), (1, 1))
        return output

## Bringing Everything Together

Now let us test if our model works.

In [27]:
import torchvision
import torch.nn.functional as F
import torch.optim as optim

In [70]:
n_epochs = 1
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [75]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=False)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=False)

In [82]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = QConv2D(1, 3, kernel_size=(3, 3))
        self.conv2 = QConv2D(3, 3, kernel_size=(3, 3))
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(12, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.max_pool2d(x, 2)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 12)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [83]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './results/model.pth')
      torch.save(optimizer.state_dict(), './results/optimizer.pth')

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [84]:
import os

if not os.path.isdir('./results'):
    os.mkdir('./results')

test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

Using previously saved contraction order at  <_io.BufferedReader name='/home/minzhaoliu/Saved_Contraction_Orders/OrderingOptimizer/QNN/n_9_l_1.pickle'>
Using previously saved contraction order at  <_io.BufferedReader name='/home/minzhaoliu/Saved_Contraction_Orders/OrderingOptimizer/QNN/n_27_l_1.pickle'>


/tmp/ipykernel_99656/3784323493.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
/home/minzhaoliu/.conda/envs/qtensor_ai/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3052, Accuracy: 980/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301146
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.296292
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.316158
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.324809


KeyboardInterrupt: 